In [1]:
from bs4 import BeautifulSoup

import pandas as pd

import numpy as np

import re

# Selenium
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.support.ui import Select

# Multi
from multiprocessing import Pool, cpu_count

# Time
import datetime
import time

# Tools
from itertools import repeat
import unittest
from unittest.mock import patch

# Bekeautocenter


In [75]:
# -------------------------------
# ---------- FONCTIONS ----------
# -------------------------------

def turn_page ():
    try:
        next_page_button = browser.find_element_by_xpath('/html/body/div[2]/div/div[1]/form/div[2]/div[2]/div[2]/div/ul/li[7]/a')
        next_page_button.click()
    except(NoSuchElementException):
        print ("Il n'y a plus de pages à tourner")
        
# dans la page de n resultats, recuperer les liens de chaque résultat -> liste de liens
def get_result_urls(sleep_time, resultats_par_page):
    url_list = []
    counter = 0
    for index in range(3,resultats_par_page+2):
        try:
            time.sleep(sleep_time)
            url_list.append(browser.find_element_by_xpath(f'/html/body/div[2]/div/div[1]/form/div[2]/div[{index}]/article/div/div[1]/div/a').get_attribute('href'))
            time.sleep(sleep_time)
            counter += 1
        except:
            print("Il n'y a plus d'url à récupérér!!!! Itérations effectuées : ", counter)
            break
    return url_list

# ouvrir url dans nouvelle fenêtre, executer get_attributes, remplir liste, fermer onglet -> liste complétée
def open_url_get_attributtes_go_back(sleep_time,liste_a_completer):
    counter = 0
    for url in url_list[0:20]:
        try:
            main_window = browser.current_window_handle
            # Open a new tab in blank
            browser.execute_script("window.open(''),'_blannk'")
            # Switch to the new window
            browser.switch_to.window(browser.window_handles[1])

            # Change the url in the .get**
            browser.get(url)
            # Completer la liste
            attributs_voiture = get_attributes(beke_occasion_xpath)
            liste_a_completer.append(attributs_voiture)
            # Temps d'attente
            time.sleep(sleep_time)
            # Close Current Tab
            browser.close()
            # Focus to the main window
            browser.switch_to.window(main_window)
            counter += 1
        
        except:
            print("Il n'y a plus d'url dans la liste. J'ai fini ! Itérations : ", counter)
            break
        
    return liste_a_completer

def get_attributes(dictionnaire):
    liste = []
    for item in dictionnaire.items() : 

                try:
                     liste.append(browser.find_element_by_xpath(item[1]).text)

                except (NoSuchElementException) : 
                    liste.append('nan')
    return liste

# VOITURES D'OCCASION SI OUVERT DANS NOUVEL L'ONGLET 
beke_occasion_xpath = {
    "nom":'/html/body/div[1]/div[1]/div/div/div[2]/h3',    
    "sous_nom":'/html/body/div[1]/div[1]/div/div/div[2]/span',
    "ref":"/html/body/div[1]/div[1]/div/div/div[3]/div[1]/div[2]/div",
    "marque":'/html/body/div[1]/div[1]/div/div/div[1]/ul/li[1]/a',
    "modele":'/html/body/div[1]/div[1]/div/div/div[1]/ul/li[2]/a',
    "categorie":'/html/body/div[1]/div[1]/div/div/div[1]/ul/li[3]/a',
    "co2":'/html/body/div[1]/div[1]/div/div/div[3]/div[2]/div/div[2]/div[2]/table/tbody/tr[1]/td[2]/div/div[2]',    
    "nb_km":"//td[preceding::th[text()[contains(.,'Kilométrage')]]]",    
    "carburant":'/html/body/div[1]/div[1]/div/div/div[3]/div[2]/div/div[2]/div[1]/div[2]/span',    
    "boite_vitesses":"//span[preceding::label[text()[contains(.,'Boite')]]]",    
    "couleur":"//td[preceding::th[text()[contains(.,'Couleur')]]]",
    "nb_portes":"//td[preceding::th[text()[contains(.,'Nombre de portes')]]]",
    "nb_places":"//td[preceding::th[text()[contains(.,'Nombre de places')]]]",
    "mise_circulation":"//td[preceding::th[text()[contains(.,'circulation')]]]",    
    "conso_mixte":"//td[preceding::th[text()[contains(.,'Conso. mixte')]]]",    
    "p_fiscale":"//td[preceding::th[text()[contains(.,'Puissance fiscale')]]]",    
    "p_din":"//td[preceding::th[text()[contains(.,'Puissance DIN')]]]",    
    "critair":"//th[text()[contains(.,'Certificat')]]",    
    "prix":'/html/body/div[1]/div[1]/div/aside/div/div[1]/div[1]/div[1]/div/span',     
}

# -------------------------------
# ---------- SCRAPPING ----------
# -------------------------------

# HEADLESS
options = webdriver.ChromeOptions()
options.add_argument('headless')
browser = webdriver.Chrome(options=options)
browser.get('http://www.bekeautocenter.fr/catalogue/?search[vnvo]=1&search[marques][0]=Citro%C3%ABn&search[marques][1]=Peugeot&search[marques][2]=Renault&search[minPrice]=0&search[maxPrice]=40000&search[order]=marque|asc&search[limit]=100&search[_token]=S90FBTV8BwR2bWGAgT4fdtttoPj4ZPTKrUYdM27skPg&_=1590561915186&page=1')


# browser = webdriver.Chrome()
# browser.maximize_window()
# browser.get('http://www.bekeautocenter.fr/catalogue/?search[vnvo]=1&search[marques][0]=Citro%C3%ABn&search[marques][1]=Peugeot&search[marques][2]=Renault&search[minPrice]=0&search[maxPrice]=40000&search[order]=marque|asc&search[limit]=100&search[_token]=S90FBTV8BwR2bWGAgT4fdtttoPj4ZPTKrUYdM27skPg&_=1590561915186&page=1')

# Nombre total de résultats au format : '816 RÉFÉRENCES' (type = str)
beke_total_results = browser.find_element_by_xpath('/html/body/div[2]/div/div[1]/form/div[1]/div[2]/h3').text
print("Nombre total de résultats :", beke_total_results)

# Dernière page disponible au format : 9 (type = int)
beke_last_page = int(re.findall("(\d+) ",beke_total_results)[0]) // 100 + 1
print("Dernière page disponible : ", beke_last_page)

liste_a_completer = []

for page in range(1,beke_last_page+1):
    
    # Liste de liens de la page en cours :
    url_list = get_result_urls(0.02, 100)

    # Va dans chaque lien et recupere les infos de chaque voiture
    liste_a_completer = open_url_get_attributtes_go_back(0.2,liste_a_completer)

    # On tourne la page :
    turn_page() 
    
    print("Echantillons recupérés : ", len(liste_a_completer))

Nombre total de résultats : 434 RÉFÉRENCES
Dernière page disponible :  5
Echantillons recupérés :  20
Il n'y a plus d'url à récupérér!!!! Itérations effectuées :  2
Echantillons recupérés :  22
Il n'y a plus d'url à récupérér!!!! Itérations effectuées :  6
Echantillons recupérés :  28
Il n'y a plus d'url à récupérér!!!! Itérations effectuées :  9
Echantillons recupérés :  37
Il n'y a plus d'url à récupérér!!!! Itérations effectuées :  6
Il n'y a plus de pages à tourner
Echantillons recupérés :  43


## Générer le DataFrame

In [74]:
# Convertir liste de listes en DataFrame
column_names = [item[0] for item in beke_occasion_xpath.items()]

df = pd.DataFrame.from_records(liste_a_completer,columns=column_names)
df_copy = df
df.shape

(50, 19)

In [64]:
df = df_copy
df.head(5)

,nom,sous_nom,ref,marque,modele,categorie,co2,nb_km,carburant,boite_vitesses,couleur,nb_portes,nb_places,mise_circulation,conso_mixte,p_fiscale,p_din,critair,prix
0,CITROËN BERLINGO,TAILLE M PURETECH 110 S S BVM6 SHINE,RÉF. 12003754 - 203453,CITROËN,BERLINGO,MONOSPACE,119,5,ESSENCE,MANUELLE,GRIS ACIER,5,5,23/08/2019,4.0,6,110,1,21366
1,CITROËN BERLINGO,TAILLE M PURETECH 110 S S BVM6 SHINE,RÉF. 12003754 - 203473,CITROËN,BERLINGO,MONOSPACE,119,8,ESSENCE,MANUELLE,GRIS ACIER,5,5,11/07/2019,4.0,6,110,1,21162
2,CITROËN BERLINGO,1.6 HDI 100 CLUB CAB. APPRO.,RÉF. 1841676461 - 1841676461,CITROËN,BERLINGO,DIVERS,113,63,DIESEL,MANUELLE,BLANC,5,5,02/03/2017,NaN,5,100,2,12728
3,CITROËN BERLINGO,NOUVEAU CABINE APPROFONDIE XL PURETECH 110 S&S...,RÉF. 2093496953 - 2093496953,CITROËN,BERLINGO,DIVERS,117,10,ESSENCE,MANUELLE,BLANC,4,5,29/01/2020,NaN,6,110,1,20228
4,CITROËN C1,VTI 72 S S FEEL,RÉF. 12001202 - 195896,CITROËN,C1,CITADINE,85,7,ESSENCE,MANUELLE,BLEU CALVI,5,4,17/05/2019,7.0,3,72,1,11676


In [71]:
def preprocess_beke_dataframe (df)
    # Enlever les espaces entre chiffres
    df['prix'] = df['prix'].str.replace(' ', '', regex=True)
    df['p_din'] = df['p_din'].str.replace(' ', '', regex=True)
    df['ref'] = df['ref'].str.replace(' - ', '', regex=True)

    # Remplacer nan par '0' pour pouvoir convertir en nombre plus tard
    df['conso_mixte'] = df['conso_mixte'].str.replace('nan', '0', regex=True)

    # Extraire par regex puis changer le type
    for col in ['co2', 'nb_km', 'nb_portes', 'nb_places', 'p_fiscale', 'p_din', 'critair', 'prix']:
        df[col] = df[col].str.extract(r'(\d+)').astype(int)

    # Extraire par regex puis changer le type (colonnes difficiles)
    df['conso_mixte'] = df['conso_mixte'].str.extract('(\d+\.?)L').astype(float)
    df['ref'] = df['ref'].str.extract('(\d+)').astype(float)
return df

In [402]:
# df.to_csv("beke_242.csv", index=False)